# E-Commerce Chatbot - Basic Implementation

This notebook implements a simple chatbot for handling product and order queries.

In [ ]:
!pip install langchain

In [45]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import requests
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory
import json


In [32]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"


In [4]:
products_df = pd.read_csv('Data/Product_Information_Dataset.csv')
orders_df = pd.read_csv('Data/Order_Data_Dataset.csv')

products_df = products_df.fillna('')
orders_df = orders_df.fillna('')

print(f"Loaded {len(products_df)} products and {len(orders_df)} orders")
print("\nProduct columns:", products_df.columns.tolist())
print("\nOrder columns:", orders_df.columns.tolist())

Loaded 5000 products and 51290 orders

Product columns: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'store', 'categories', 'details', 'parent_asin']

Order columns: ['Order_Date', 'Time', 'Aging', 'Customer_Id', 'Gender', 'Device_Type', 'Customer_Login_type', 'Product_Category', 'Product', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost', 'Order_Priority', 'Payment_method']


In [56]:
class BaseLanguageModel:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model = model_name
        self.api_key = os.getenv("OPENAI_API_KEY")
        
    def generate_response(self, prompt: str, functions=None) -> str:
        messages = [{"role": "user", "content": prompt}]
        
        try:
            data = {
                "model": self.model,
                "messages": messages,
                "temperature": 0.7
            }
            
            if functions:
                data["functions"] = functions
                data["function_call"] = "auto"
                
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json=data
            )
            return response.json()
        except Exception as e:
            return f"Error generating response: {str(e)}"

In [77]:
class EcommerceBot:
    def __init__(self, products_df, orders_df):
        self.products_df = products_df
        self.orders_df = orders_df
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.language_model = BaseLanguageModel()
        self.memory = ConversationBufferMemory()
        
        # Compute product embeddings once
        print("Computing product embeddings...")
        self.product_embeddings = self.embedding_model.encode(
            (products_df['title'] + ' ' + products_df['description'].fillna('')).tolist()
        )
        print("Embeddings computed!")

    def process_query(self, query):
        history = self.memory.load_memory_variables({})
        
        # Analysis prompt
        analysis_prompt = f"""Analyze this query and conversation history:
        History: {history.get('history', '')}
        Current query: {query}"""
                
        # First, analyze the query and history
        analyze_response = self.language_model.generate_response(
            analysis_prompt,
            functions=[{
                "name": "analyze_query",
                "description": "Analyze the query type and extract customer ID if present",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query_type": {
                            "type": "string",
                            "enum": ["product", "order"]
                        },
                        "customer_id": {
                            "type": "integer",
                            "description": "Customer ID if mentioned in current query or found in conversation history"
                        }
                    },
                    "required": ["query_type"]
                }
            }]
        )
    
        # Extract query type and customer ID
        if 'function_call' in analyze_response['choices'][0]['message']:
            analysis = json.loads(analyze_response['choices'][0]['message']['function_call']['arguments'])
            query_type = analysis.get('query_type')
            customer_id = analysis.get('customer_id')
            
            if query_type == "order" and not customer_id:
                return "Could you please provide your customer ID?"
            
            # Get relevant information based on query type
            if query_type == "order":
                context = self._get_order_info(customer_id)
            else:
                # For mixed queries, include both product and order context if we have customer_id
                context = self._get_product_info(query)
                if customer_id:
                    order_context = self._get_order_info(customer_id)
                    if order_context:
                        context += "\n\nRelated Order Information:\n" + order_context
        else:
            context = self._get_product_info(query)
    
        # Final response prompt
        response_prompt = f"""You are an e-commerce assistant. Use this information to respond:
        
        Context: {context}
        History: {history.get('history', '')}
        Query: {query}
        
        If this is an order query and you have customer information, make sure to mention the customer ID in your response.
        When suggesting products, always include all available information including ratings and prices.
        If referring to orders, include all order details available in the context."""
    
        response = self.language_model.generate_response(response_prompt)
        if isinstance(response, dict) and 'choices' in response:
            response = response['choices'][0]['message']['content']
            
        self.memory.save_context({"input": query}, {"output": response})
        return response
    def _get_product_info(self, query):
        query_embedding = self.embedding_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.product_embeddings)[0]
        top_indices = similarities.argsort()[-3:][::-1]
        
        results = []
        for idx in top_indices:
            product = self.products_df.iloc[idx]
            results.append(f"""
            Product: {product['title']}
            Price: ${product['price']}
            Rating: {product['average_rating']} stars
            Description: {product['description']}
            """)
        return "\n".join(results)

    def _get_order_info(self, customer_id):
        try:
            response = requests.get(f"http://127.0.0.1:8000/data/customer/{customer_id}")
            orders = response.json()
            if isinstance(orders, list):
                formatted_orders = []
                for order in orders:
                    formatted_orders.append(f"""
                    Order Date: {order['Order_Date']}
                    Product Category: {order['Product_Category']}
                    Sales Amount: ${order['Sales']}
                    Shipping Cost: ${order['Shipping_Cost']}
                    Status: {order['Order_Priority']}
                    Customer ID: {customer_id}
                    Product: {order['Product']}
                    """)
                return "Customer Orders:\n" + "\n".join(formatted_orders)
            return "No orders found"
        except Exception as e:
            return f"Error fetching orders: {str(e)}"

In [78]:
bot = EcommerceBot(products_df, orders_df)

Computing product embeddings...
Embeddings computed!


In [59]:
def view_memory(bot):
    print("\nConversation History:")
    print("-" * 50)
    print(bot.memory.load_memory_variables({}).get('history', 'No history'))
    print("-" * 50)

def clear_memory(bot):
    bot.memory.clear()
    print("Memory cleared!")

In [85]:
clear_memory(bot)

Memory cleared!


In [95]:
def run_test_suite(bot):
    clear_memory(bot)
    print("Test Suite Beginning...")
    
    # Test Sequence 1: Basic Product Queries
    print("\n1. Product Information Query")
    print("Human: What are some good guitar strings for beginners?")
    print("Bot:", bot.process_query("What are some good guitar strings for beginners?"))
    print('-' * 50)
    
    print("\n2. Follow-up Product Question")
    print("Human: Which one has the best rating?")
    print("Bot:", bot.process_query("Which one has the best rating?"))
    print('-' * 50)
    
    print("\n3. Specific Product Feature Question")
    print("Human: Tell me more about the Martin strings")
    print("Bot:", bot.process_query("Tell me more about the Martin strings"))
    print('-' * 50)
    
    # Test Sequence 2: Order Queries
    print("\n4. Order Status Without ID")
    print("Human: I want to check my order status")
    print("Bot:", bot.process_query("I want to check my order status"))
    print('-' * 50)
    
    print("\n5. Providing Customer ID")
    print("Human: My customer ID is 41066")
    print("Bot:", bot.process_query("My customer ID is 41066"))
    print('-' * 50)
    
    print("\n6. Follow-up Order Question")
    print("Human: What was my most recent order?")
    print("Bot:", bot.process_query("What was my most recent order?"))
    print('-' * 50)
    
    # Test Sequence 3: Mixed Queries
    print("\n7. Follow-up Order Question")
    print("Human: Did I have any other orders?")
    print("Bot:", bot.process_query("Did I have any other orders?"))
    print('-' * 50)
    
    print("\n8. Product recommendations based on Order")
    print("Human: Do you have any products similar to what I ordered?")
    print("Bot:", bot.process_query("Do you have any products similar to what I ordered?"))
    print('-' * 50)
    
    print("\n9. Price Comparison")
    print("Human: How does the price compare to other similar products?")
    print("Bot:", bot.process_query("How does the price compare to other similar products?"))
    print('-' * 50)
    
    print("\n10. Complex Context Query")
    print("Human: Based on my order history and the current products, what would you recommend?")
    print("Bot:", bot.process_query("Based on my order history and the current products, what would you recommend?"))
    
    # Clear memory after tests
    clear_memory(bot)


In [96]:
run_test_suite(bot)

Memory cleared!
Test Suite Beginning...

1. Product Information Query
Human: What are some good guitar strings for beginners?
Bot: For beginners looking for good guitar strings, I recommend the **Martin M130 80-20 Silk & Steel Strings**. Here are the details:

- **Price:** $13.87
- **Rating:** 4.5 stars
- **Description:** Martin Strings are high-quality strings for daily use. Core and wrap wires must meet strict requirements to make the grade. Martin Strings are wound to precise specifications.

These strings are well-regarded for their quality and are a great choice for those starting out.

If you're looking for a complete beginner guitar kit, you might also consider the **Best Choice Products 38in Beginner All Wood Acoustic Guitar Starter Kit**:

- **Price:** $49.99
- **Rating:** 4.2 stars
- **Description:** This kit includes a case, strap, digital tuner, pick, and strings, making it a comprehensive choice for new guitarists.

Let me know if you need any further assistance or have an